In [10]:
#!/usr/bin/env python3
"""
Create “assist” CSV files by shifting + interpolating simulation data onto
the experimental x‑grid.

Directory layout
----------------
SIM_results/NH3_NP_CF_reduce/<CASE>.csv   – simulation files
EXP_data/NH3_NP_CF_EXP/<CASE>.csv         – experimental files
SIM_results/NH3_NP_CF_assist/<CASE>.csv   – output (created)

Required columns
  • simulation : 'grid'           (mm, 0 = nozzle exit)  + species
  • experiment : 'x' (or first)   (mm, 0 = datum)        + anything

Only the simulation ‘grid’ is shifted before 1‑D interpolation; the
experimental x‑values are left untouched.
"""

from pathlib import Path
from typing import Dict, Sequence, Optional, Tuple

import numpy as np
import pandas as pd

# --------------------------------------------------------------------------
# 1.  Configuration
# --------------------------------------------------------------------------
SIM_DIR = Path("SIM_results/NH3_NP_CF_reduce")
EXP_DIR = Path("EXP_data/NH3_NP_CF_EXP")
OUT_DIR = Path("SIM_results/NH3_NP_CF_assist")
OUT_DIR.mkdir(parents=True, exist_ok=True)

SIM_TO_MM = 1000.0          # 1 m = 1000 mm  (set to 1.0 if your sim grid is already mm)


ASSIST_SPECIES: Sequence[str] = (
    # scalar / thermodynamic first
    "T", "density",
    # species (bare names – prefix detected automatically)
    "NO", "NH3", "H2", "O2", "H", "O", "OH", "HO2", "H2O", "H2O2",
    "NH2", "NH", "N", "NNH",
    "NH2OH", "H2NO", "HNOH", "HNO", "HON",
    "NO2", "HONO", "HNO2", "NO3", "HONO2",
    "N2O", "N2H4", "N2H3", "N2H2", "H2NN",
    "N2", "AR",
)

# axial shifts (mm) applied to SIM grid **before** interpolation
SIM_SHIFT: Dict[str, float] = {
    # ── nitrogen‑only series ───────────────────────────────────────────────
    "N_0":  -4.40,
    "N_08": -4.50,
    "N_16": -4.65,
    "N_24": -4.76,
    "N_30": -4.85,
    # ── AH series ──────────────────────────────────────────────────────────
    "AH82":      -4.90,
    "AH64":      -4.45,
    "AH46":      -4.35,
    "AH64_K80":  -4.40,
    "AH64_K120": -4.75,
    "AH64_K140": -4.85,
}

# --------------------------------------------------------------------------
# 2.  Helpers
# --------------------------------------------------------------------------
def _load_case(case: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Return (exp_df, sim_df)."""
    return (
        pd.read_csv(EXP_DIR / f"{case}.csv"),
        pd.read_csv(SIM_DIR / f"{case}.csv"),
    )


def _find_sim_column(df: pd.DataFrame, name: str) -> Optional[str]:
    """
    Return the column in *df* that holds *name*.

      • exact match ('T', 'density', …)
      • else 'X_name' or 'Y_name' (first found)

    Returns None if the column is absent (caller may skip it).
    """
    if name in df.columns:
        return name
    for prefix in ("X_", "Y_"):
        col = f"{prefix}{name}"
        if col in df.columns:
            return col
    return None


def _interp_1d(x_src, y_src, x_new) -> np.ndarray:
    """1‑D linear interpolation with NaN outside the source domain."""
    return np.interp(x_new, x_src, y_src, left=np.nan, right=np.nan)

# --------------------------------------------------------------------------
# 3.  Main loop
# --------------------------------------------------------------------------
for sim_file in SIM_DIR.glob("*.csv"):
    case = sim_file.stem                           # e.g. 'N_08'
    if not (EXP_DIR / f"{case}.csv").exists():
        print(f"[!] experimental file for '{case}' not found – skipped")
        continue

    exp_df, sim_df = _load_case(case)

    # coordinates
    x_exp_col = "x" if "x" in exp_df.columns else exp_df.columns[0]
    x_sim_col = "grid" if "grid" in sim_df.columns else sim_df.columns[0]

    x_exp = exp_df[x_exp_col].to_numpy(copy=True)
    x_sim = sim_df[x_sim_col].to_numpy(copy=True) * SIM_TO_MM

    # --- shift the simulation grid only -------------------------------------
    x_sim += SIM_SHIFT.get(case, 0.0)              # default: no shift

    # --- build output -------------------------------------------------------
    assist: Dict[str, np.ndarray] = {x_exp_col: x_exp}

    for name in ASSIST_SPECIES:
        sim_col = _find_sim_column(sim_df, name)
        if sim_col is None:
            print(f"[!] {case}: '{name}' not in simulation file – skipped")
            continue
        assist[sim_col] = _interp_1d(x_sim,
                                     sim_df[sim_col].to_numpy(),
                                     x_exp)

    pd.DataFrame(assist).to_csv(OUT_DIR / f"{case}.csv", index=False)
    print(f"[✓] {case:10s} → {OUT_DIR / (case + '.csv')}")

print("\nAll assist files created successfully.")

[✓] AH64       → SIM_results/NH3_NP_CF_assist/AH64.csv
[✓] AH64_K80   → SIM_results/NH3_NP_CF_assist/AH64_K80.csv
[✓] AH46       → SIM_results/NH3_NP_CF_assist/AH46.csv
[✓] AH64_K120  → SIM_results/NH3_NP_CF_assist/AH64_K120.csv
[✓] N_0        → SIM_results/NH3_NP_CF_assist/N_0.csv
[✓] N_30       → SIM_results/NH3_NP_CF_assist/N_30.csv
[✓] AH82       → SIM_results/NH3_NP_CF_assist/AH82.csv
[✓] AH64_K140  → SIM_results/NH3_NP_CF_assist/AH64_K140.csv
[✓] N_24       → SIM_results/NH3_NP_CF_assist/N_24.csv
[✓] N_16       → SIM_results/NH3_NP_CF_assist/N_16.csv
[✓] N_08       → SIM_results/NH3_NP_CF_assist/N_08.csv

All assist files created successfully.


# Interpolate EXP to SIM

In [11]:
#!/usr/bin/env python3
"""
Create “reverse‑assist” CSV files by interpolating EXP data onto the
(unshifted) SIM grid.  Outside the EXP domain the original SIM values are kept.

Output: SIM_results/NH3_NP_CF_reverse/<CASE>.csv
"""
from pathlib import Path
from typing import Dict, Sequence, Optional, Tuple

import numpy as np
import pandas as pd

# ------------------------------------------------------------------------- #
# 1.  Configuration                                                         #
# ------------------------------------------------------------------------- #
SIM_DIR   = Path("SIM_results/NH3_NP_CF_reduce")
EXP_DIR   = Path("EXP_data/NH3_NP_CF_EXP")
OUT_DIR   = Path("EXP_data/NH3_NP_CF_EXP_assist")
OUT_DIR.mkdir(parents=True, exist_ok=True)

SIM_TO_MM = 1000.0          # 1 m → 1000 mm  (set to 1 if sim grid already mm)

# x,X_NO,T,X_O2,X_N2,X_NH3,X_H2O,X_H2
ASSIST_SPECIES: Sequence[str] = (
    "NO", "NH3", "H2", "O2", "N2", "H2O",
)

# axial shifts (mm) originally used for SIM → EXP interpolation
SIM_SHIFT: Dict[str, float] = {
    "N_0": -4.40, "N_08": -4.50, "N_16": -4.65, "N_24": -4.76, "N_30": -4.85,
    "AH82": -4.90, "AH64": -4.45, "AH46": -4.35,
    "AH64_K80": -4.40, "AH64_K120": -4.75, "AH64_K140": -4.85,
}

# ------------------------------------------------------------------------- #
# 2.  Helpers                                                               #
# ------------------------------------------------------------------------- #
def _load_case(case: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Return (exp_df, sim_df)."""
    return (
        pd.read_csv(EXP_DIR / f"{case}.csv"),
        pd.read_csv(SIM_DIR / f"{case}.csv"),
    )


def _find_column(df: pd.DataFrame, base: str) -> Optional[str]:
    """
    Find *base* in df:

      • exact match ('T', 'density', …)
      • or 'X_base' / 'Y_base' (first found)

    Returns None if absent.
    """
    if base in df.columns:
        return base
    for p in ("X_", "Y_"):
        col = f"{p}{base}"
        if col in df.columns:
            return col
    return None


def _interp_or_keep(x_exp, y_exp, x_sim, y_sim) -> np.ndarray:
    """Interpolate exp→sim; keep sim where exp is NaN."""
    y_new = np.interp(x_sim, x_exp, y_exp, left=np.nan, right=np.nan)
    if np.isnan(y_new).any():
        mask = np.isnan(y_new)
        y_new[mask] = y_sim[mask]
    return y_new


# ------------------------------------------------------------------------- #
# 3.  Main loop                                                             #
# ------------------------------------------------------------------------- #
for sim_file in SIM_DIR.glob("*.csv"):
    case = sim_file.stem
    exp_file = EXP_DIR / f"{case}.csv"
    if not exp_file.exists():
        print(f"[!] EXP file for '{case}' not found – skipped")
        continue

    exp_df, sim_df = _load_case(case)

    # grids
    x_exp_col = "x" if "x" in exp_df.columns else exp_df.columns[0]
    x_sim_col = "grid" if "grid" in sim_df.columns else sim_df.columns[0]

    x_sim = sim_df[x_sim_col].to_numpy(copy=True) * SIM_TO_MM
    x_exp = exp_df[x_exp_col].to_numpy(copy=True)

    # shift EXP grid by the opposite of the SIM shift
    x_exp += -SIM_SHIFT.get(case, 0.0)

    # build output on the SIM grid
    reverse: Dict[str, np.ndarray] = {x_sim_col: x_sim}

    for base in ASSIST_SPECIES:
        exp_col = _find_column(exp_df, base)
        sim_col = _find_column(sim_df, base)

        if exp_col is None and sim_col is None:
            print(f"[!] {case}: '{base}' in neither file – skipped")
            continue
        if exp_col is None:           # no experimental data → keep SIM
            reverse[sim_col] = sim_df[sim_col].to_numpy()
            continue
        if sim_col is None:           # interpolate, no fallback needed
            reverse_name = exp_col    # keep the EXP prefix (X_/Y_/scalar)
            reverse[reverse_name] = np.interp(
                x_sim, x_exp, exp_df[exp_col].to_numpy(),
                left=np.nan, right=np.nan
            )
            continue

        # both columns exist → interpolate + fallback
        reverse[sim_col] = _interp_or_keep(
            x_exp, exp_df[exp_col].to_numpy(),
            x_sim, sim_df[sim_col].to_numpy()
        )

    pd.DataFrame(reverse).to_csv(OUT_DIR / f"{case}.csv", index=False)
    print(f"[✓] {case:10s} → {OUT_DIR / (case + '.csv')}")

print("\nAll reverse‑assist files created successfully.")

[✓] AH64       → EXP_data/NH3_NP_CF_EXP_assist/AH64.csv
[✓] AH64_K80   → EXP_data/NH3_NP_CF_EXP_assist/AH64_K80.csv
[✓] AH46       → EXP_data/NH3_NP_CF_EXP_assist/AH46.csv
[✓] AH64_K120  → EXP_data/NH3_NP_CF_EXP_assist/AH64_K120.csv
[✓] N_0        → EXP_data/NH3_NP_CF_EXP_assist/N_0.csv
[✓] N_30       → EXP_data/NH3_NP_CF_EXP_assist/N_30.csv
[✓] AH82       → EXP_data/NH3_NP_CF_EXP_assist/AH82.csv
[✓] AH64_K140  → EXP_data/NH3_NP_CF_EXP_assist/AH64_K140.csv
[✓] N_24       → EXP_data/NH3_NP_CF_EXP_assist/N_24.csv
[✓] N_16       → EXP_data/NH3_NP_CF_EXP_assist/N_16.csv
[✓] N_08       → EXP_data/NH3_NP_CF_EXP_assist/N_08.csv

All reverse‑assist files created successfully.
